<a href="https://colab.research.google.com/github/demetriusohalloran-byte/DevOps/blob/main/MSAI631_groupproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Install the necessary library
!pip install python-telegram-bot python-dotenv

print("Installed 'python-telegram-bot' and 'python-dotenv'.")

<frozen posixpath>:82: RuntimeWarning: coroutine 'main_loop' was never awaited


Installed 'python-telegram-bot' and 'python-dotenv'.


Next, we'll create the bot's code. Remember to replace `YOUR_BOT_TOKEN` in the `.env` file with your actual Telegram bot token. If you don't have one, you can get it from [@BotFather](https://t.me/botfather) on Telegram.

This bot will:
1.  Respond to the `/start` command with a welcome message.
2.  Echo any text message it receives.

In [ ]:
import os
from dotenv import load_dotenv
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import logging
import nest_asyncio # Import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load environment variables
load_dotenv()
BOT_TOKEN = os.getenv("BOT_TOKEN")

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger(__name__)

# --- Command Handlers ---
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Sends a welcome message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}! I'm a simple echo bot.",
    )

async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Echoes the user message."""
    await update.message.reply_text(update.message.text)

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

# --- Main function to run the bot ---
def main() -> None:
    """Start the bot."""
    if not BOT_TOKEN:
        logger.error("BOT_TOKEN not found in .env. Please set it.")
        print("Error: BOT_TOKEN not found. Please update your .env file.")
        return

    # Create the Application and pass your bot's token.
    application = Application.builder().token(BOT_TOKEN).build()

    # Register command handlers
    application.add_handler(CommandHandler("start", start))

    # Register message handler
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

    # Register error handler
    application.add_error_handler(error_handler)

    # Run the bot until the user presses Ctrl-C
    print("Bot started. Send /start or any text message to your bot. Press Ctrl+C to stop.")
    application.run_polling(allowed_updates=Update.ALL_TYPES)

if __name__ == "__main__":
    main()

Bot started. Send /start or any text message to your bot. Press Ctrl+C to stop.


This code defines a simple Telegram bot. To run it:

1.  **Make sure your `.env` file has `BOT_TOKEN=YOUR_BOT_TOKEN` replaced with your actual token.** If you are using the same `.env` file from the previous task, ensure it is updated.
2.  **Execute the above code cell.** The bot will start and listen for messages.
3.  **Interact with your bot on Telegram!** Send `/start` or any text message.
4.  To stop the bot, **interrupt the execution of the cell** (e.g., by clicking the square stop button next to the running cell in Colab).

In [24]:
import os
import re
import time
import logging
import random
import requests

from urllib.parse import quote_plus
from dotenv import load_dotenv

load_dotenv()  # Load env vars from a .env file if present

# === Configuration ===

BOT_TOKEN = os.getenv("BOT_TOKEN", "YOUR_BOT_TOKEN")
CHAT_IDS = os.getenv("CHAT_IDS", "YOUR_CHAT_IDS").split(",")  # comma separated list

QUOTES_FILE = os.getenv("QUOTES_FILE", "quotes.txt")
INDEX_FILE = os.getenv("INDEX_FILE", "current_index.txt")

MAX_RETRIES = 3
RETRY_DELAY = 5  # seconds

# Emojis to rotate in header
EMOJI_OPTIONS = ["🌟", "🔥", "✨", "💡", "🌈", "⚡"]

# === Setup logging ===

logging.basicConfig(
    filename='telegram_quote_bot.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# === Functions ===

def load_quotes(filename):
    if not os.path.exists(filename):
        logging.error(f"Quotes file '{filename}' not found.")
        return []
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
    quotes = [line.strip() for line in lines if line.strip()]
    return quotes

def load_index(filename):
    if not os.path.exists(filename):
        return 0
    with open(filename, "r") as f:
        try:
            return int(f.read().strip())
        except ValueError:
            logging.warning("Invalid index file content. Resetting index to 0.")
            return 0

def save_index(filename, idx):
    with open(filename, "w") as f:
        f.write(str(idx))

def escape_markdown_v2(text):
    escape_chars = r'_*[]()~`>#+-=|{}.!'
    return re.sub(f'([{re.escape(escape_chars)}])', r'\\\1', text)

def send_telegram_message(token, chat_id, text):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": text,
        "parse_mode": "MarkdownV2",
        "disable_web_page_preview": True
    }
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = requests.post(url, data=payload, timeout=10)
            response.raise_for_status()
            logging.info(f"Message sent to chat_id {chat_id}")
            return True, response.text
        except requests.RequestException as e:
            logging.warning(f"Attempt {attempt} failed for chat_id {chat_id}: {e}")
            if attempt < MAX_RETRIES:
                time.sleep(RETRY_DELAY)
            else:
                return False, str(e)

def main():
    quotes = load_quotes(QUOTES_FILE)
    if not quotes:
        logging.error("No quotes found. Exiting.")
        return

    current_index = load_index(INDEX_FILE)
    if current_index >= len(quotes):
        current_index = 0

    quote_raw = quotes[current_index]
    quote_escaped = escape_markdown_v2(quote_raw)

    emoji = random.choice(EMOJI_OPTIONS)
    message = f"{emoji} *Inspiration of the Day:*\n\n_{quote_escaped}_"

    all_success = True
    for chat_id in CHAT_IDS:
        success, response = send_telegram_message(BOT_TOKEN, chat_id.strip(), message)
        if success:
            logging.info(f"Sent quote #{current_index + 1} to chat {chat_id}")
        else:
            logging.error(f"Failed to send quote to chat {chat_id}: {response}")
            all_success = False

    if all_success:
        save_index(INDEX_FILE, current_index + 1)
        logging.info(f"Incremented quote index to {current_index + 1}")
    else:
        logging.warning("Not incrementing index due to send failure.")

if __name__ == "__main__":
    main()


ERROR:root:Failed to send quote to chat your_chat_id_here: 404 Client Error: Not Found for url: https://api.telegram.org/botyour_bot_token_here/sendMessage


In [27]:
import os
from dotenv import load_dotenv

# Ensure we are in the correct directory where .env is located
expected_dir = 'telegram-quote-bot-py'
current_dir = os.path.basename(os.getcwd())

if current_dir != expected_dir:
    os.chdir(f'/content/{expected_dir}')
    print(f"Changed current directory to: {os.getcwd()}")

# Reload environment variables to ensure the latest .env is read
load_dotenv(override=True)

BOT_TOKEN = os.getenv("BOT_TOKEN")

if BOT_TOKEN:
    print(f"The BOT_TOKEN currently loaded is: {BOT_TOKEN}")
    if BOT_TOKEN == "your_bot_token_here":
        print("Warning: The BOT_TOKEN is still the placeholder. Please re-check your .env file.")
else:
    print("Error: BOT_TOKEN is not set in the environment.")

The BOT_TOKEN currently loaded is: 8568753476:AAEHQ_Ec38haXYIOE641C5iuZpq5wjcC0tQ
